In [2]:
import tensorflow as tf
#import matplotlib.pyplot as plt

# network 设计

In [3]:


"""
第一层为卷积层, 卷积层_1的参数  
input:  filter: 
        size: 5*5;  number: 6  channel:  1  
output: feature map: 
        size 28*28, number 6; channel :1
"""
CONV1_FILTER_SIZE = 5
CONV1_FILTER_CHANNEL = 1
CONV1_FILTER_NUM = 32

"""
第三层为卷积层, 卷积层_3的参数
input: filter:
       size 5 * 5   number 16 channel: 1
output: feature map
       size 10 * 10 number 16  channel: 1 
       
"""
CONV3_FILTER_SIZE = 5
CONV3_FILTER_CHANNEL = 1
CONV3_FILTER_NUM = 64

"""
第五层是一个全连接层, 

"""
FC5_NODES = 512


"""第六层也是一个全连接层
"""
FC6_NODES = 10

# LeNet_inference

def LeNet_inference(inTensor):
    
    """conv 1.  第一层为卷积层,有6个filter
    input size: 28 * 28 * 1
    output size: 28 * 28 * 32
    """
    with tf.variable_scope("layer1-conv1"):
        conv1_filter = tf.get_variable(name="c1_weights", 
                                       shape=[CONV1_FILTER_SIZE, CONV1_FILTER_SIZE,CONV1_FILTER_CHANNEL, CONV1_FILTER_NUM],
                                       dtype= tf.float32, 
                                       initializer = tf.truncated_normal_initializer(stddev=0.1))
        conv1 = tf.nn.conv2d(inTensor, conv1_filter, strides=[1, 1, 1, 1], padding='SAME', name="c1_op")
        conv1_bias = tf.get_variable(name="c1_bias", 
                                     shape=[CONV1_FILTER_NUM], 
                                     dtype=tf.float32, 
                                     initializer=tf.truncated_normal_initializer())
        relu_1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_bias))
    

    """pooling 2.  
     第2层为pooling层,与原始LetNet不同.原始的LeNet的pooling层有两个可训练参数
     input size: 28 * 28 * 32
     output size: 14 * 14 * 32
    """
    with tf.variable_scope("layer2-pooling1"):
        pooling2 = tf.nn.max_pool(relu_1, ksize=[1,2,2,1], strides=[1, 2, 2 ,1], padding="SAME", name="p2_op")

    
    
    """ conv 3  
    第3层为卷积层,共16个filter. 与原始LeNet不同,原始的LeNet是部分连接
    这里为了方便是,整体连接,详细情况见论文希捷
    input size 14 * 14 * 32
    out putsize 14 * 14 * 64
    """
    with tf.variable_scope("layer3-conv3"):
        conv3_filter = tf.get_variable(name="c3_weights", 
                                   shape=[CONV3_FILTER_SIZE, CONV3_FILTER_SIZE,CONV1_FILTER_NUM, CONV3_FILTER_NUM],
                                   dtype= tf.float32, 
                                   initializer = tf.truncated_normal_initializer(stddev=0.1))

        conv3_bias = tf.get_variable(name="c3_bias",
                                    shape=[CONV3_FILTER_NUM],
                                    dtype=tf.float32,
                                    initializer = tf.truncated_normal_initializer())

        conv3 = tf.nn.conv2d(pooling2, conv3_filter, strides=[1,1,1,1], padding='SAME',name="c3_op")
        relu_3 =tf.nn.relu(tf.nn.bias_add(conv3, conv3_bias))
    
    
    """pooling 4  
    第4层为pooling层, 与pooling 2 相同
    input size:  14 * 14 * 64
    output size: 7 * 7 * 64
    """
    with tf.variable_scope("layer4-conv4"):
        pooling4 = tf.nn.max_pool(relu_3, ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME", name="p4_op")

    
    """fc 5: layer-5 is a fully connected layer
    第5层为全连接层, 本层具有512个node, 输入为 7 * 7 * 64 = 3136个单元, 
    input size: 3136
    output size: 512
    """
    #stretch the input to a vector
    with tf.variable_scope("layer5-conv5"):
        input_shape = pooling4.get_shape().as_list()
        input_5 = tf.reshape(pooling4, [input_shape[0], input_shape[1]*input_shape[2]*input_shape[3]])


        fc5_weights = tf.get_variable(name = "fc5_weights", 
                                      shape = [input_shape[1]*input_shape[2]*input_shape[3], FC5_NODES],
                                      dtype = tf.float32,
                                      initializer = tf.truncated_normal_initializer(stddev=0.1))
        fc5_bias = tf.get_variable(name = "fc5_bias", 
                                   shape = [FC5_NODES],
                                   dtype = tf.float32,
                                   initializer = tf.truncated_normal_initializer())

        relu_5 = tf.nn.relu(tf.nn.bias_add(tf.matmul(input_5, fc5_weights), fc5_bias), name="fc5_op")

    
    
    """fc 6 layer 6 is a  fully connected layer
    input size 512
    output size 10
    """
    with tf.variable_scope("layer6-conv6"):
        fc6_weights = tf.get_variable(name = "fc6_weights", 
                                      shape = [FC5_NODES, FC6_NODES],
                                      dtype = tf.float32,
                                     initializer = tf.truncated_normal_initializer(stddev=0.1))
        fc6_bias = tf.get_variable(name = "fc6_bias",
                                   shape = [FC6_NODES],
                                   dtype = tf.float32,
                                   initializer = tf.truncated_normal_initializer())
        relu_6 = tf.nn.bias_add(tf.matmul(relu_5, fc6_weights),fc6_bias, name = "fc6_op")
 
    
    """最后的输出单元
    softmax分类器
    """
    #logits = tf.nn.softmax(relu_6)
    #result = tf.one_hot(tf.argmax(tf.transpose(logits)), 10)
    return relu_6
    
  

